In [4]:
import os

# Define the base directory (change this to your target directory)
base_directory = r"E:"  # Replace with your actual path
# List to store file paths
file_list = []

# Walk through all directories and subdirectories
for root, _, files in os.walk(base_directory):
    for file in files:
        if file.endswith(".csv") or file.endswith(".xlsx") or file.endswith(".xls"):
            file_list.append((root, file))

# Total file count
total_files = len(file_list)




In [5]:
import os

base_directory = base_directory

# List only directories (folders) in the base directory
folders = [f for f in os.listdir(base_directory) if os.path.isdir(os.path.join(base_directory, f))]

# Print all folders
print("Folders in base directory:", folders)


Folders in base directory: ['.git', 'database_training', 'email_creation', 'email_verifier', 'files', 'helper', 'old_database_files', 'static', 'templates', 'uploads']


In [ ]:

# Print file names with remaining count
for index, (folder, filename) in enumerate(file_list, start=1):
    remaining_files = total_files - index
    print(f"File {index}/{total_files}: {filename} (Remaining: {remaining_files}) in {folder}")

print(f"\nTotal CSV/Excel files found: {total_files}")

In [7]:
# !pip install pandas÷

In [1]:
from database_training.database_process_mutifile import process_files_and_save_output
from database_training.merger_for_pckl import update_pkl_with_csv
import pandas as pd
from database_training.update_db_manually import update_pickle
from email_creation.create_emails import email_creator_app
from database_training.database_process_single_file import process_single_file


In [ ]:
        files = request.files.getlist('folder')  # Get all files in the folder
        filenames = [file.filename for file in files if file.filename != '']
        for i in filenames:
            print ("filenames ",i)
            try:
                if i.endswith('.csv') or i.endswith('.xlsx'):
                    file_path = f"{base_path}//{i}"

                    process_single_file(file_path, Output_files_path)

            except Exception as e:
                log_message = f"Error processing file: {str(e)}"
        update_pkl_with_csv(Output_files_path, db_pickle_file_path)